In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GPU-Accelerated PySpark") \
    .config("spark.executor.resource.gpu.amount", "1") \
    .config("spark.task.resource.gpu.amount", "1") \
    .config("spark.executorEnv.PYTHONPATH", "/path/to/cudf/python") \
    .config("spark.executor.memory", "48G") \
    .config("spark.executor.cores", "8") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.driver.memory", "16G") \
    .config("spark.master", "local[8]") \
    .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
25/01/17 10:11:31 WARN Utils: Your hostname, DESKTOP-5OQCI6I resolves to a loopback address: 127.0.1.1; using 172.17.107.8 instead (on interface eth0)
25/01/17 10:11:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/17 10:11:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/17 10:11:33 WARN RapidsPluginUtils: RAPIDS Accelerator 24.12.0 using cudf 24.12.0, private revision 874fe2a69931879dc50faad111280e8ca75467cc
25/01/17 10:11:33 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
25/01/17 10:11:33 WARN RapidsPluginUtils: spark.rapids.sql.explain is set to `NOT_ON_GPU`. Set it to 'NONE' to suppress the diagnostics logging about the

In [11]:
!pwd

/mnt/d/Spark_work


In [10]:
import cudf
print(cudf.Series([1, 2, 3]))

CudaSupportError: Error at driver init: 
Call to cuInit results in CUDA_ERROR_NO_DEVICE (100):

In [7]:
import cudf
print(cudf.__version__)

24.12.00


In [10]:
import rmm
import cudf

rmm.reinitialize(pool_allocator=True, initial_pool_size=2**30)  # 1GB pool
cudf.set_allocator(rmm.mr.PoolMemoryResource)
print(cudf.Series([1, 2, 3]))


MemoryError: std::bad_alloc: out_of_memory: RMM failure at:/tmp/pip-build-env-9oax9jdp/normal/lib/python3.10/site-packages/librmm/include/rmm/mr/device/pool_memory_resource.hpp:276: Maximum pool size exceeded

In [20]:
def run_test(file_path):
    start_time = time.time()
    
    # Load CSV file
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    
    firstvalue = df.first()['SICCode_SicText_1']
    # Perform some transformations
    count_occurrences = df.filter(df.name == firstvalue).count()
    
    # Trigger an action
    print(count_occurrences)  # or transformed_df.write.csv("output_path")
    
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.2f} seconds")

# Replace with your CSV file path
csv_file_path = "./speed_test/BasicCompanyData-2025-01-01-part1_7.csv"

run_test(csv_file_path)

spark.stop()

ERROR:root:KeyboardInterrupt while sending command.][Stage 2:>    (0 + 0) / 1]
Traceback (most recent call last):
  File "/home/trio/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/trio/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 